In [2]:
import torch

opt_act_path = 'loss_quantification/opt_125m_acts'
quant_act_path = 'loss_quantification/opt_125m_q4_256g_acts'

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch.nn as nn
from transformers.pytorch_utils import Conv1D

opt_model_id = 'facebook/opt-125m'

model = AutoModelForCausalLM.from_pretrained(opt_model_id)


/home/jwjeong/anaconda3/envs/autogptq/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
orig = torch.load(opt_act_path+'/'+'model.decoder.layers.0.self_attn.out_proj.pt')
q = torch.load(quant_act_path+'/'+'model.decoder.layers.0.self_attn.out_proj.pt')

In [18]:
import torch.nn.functional as F

avg_mse = {}
for name, m in model.named_modules():
    if isinstance(m, nn.Linear) | isinstance(m, Conv1D):
        if name.split('.')[-1] != 'lm_head':
            orig = torch.load(opt_act_path+'/'+name+'.pt')
            quant = torch.load(quant_act_path+'/'+name+'.pt')
            sum = 0
            for i in range(len(orig)):
                mse_loss = F.mse_loss(orig[i], quant[i])
                sum += mse_loss

            avg_mse[name] = sum / len(orig)

In [20]:
total_loss = 0
for key in avg_mse:
    total_loss += avg_mse[key]

In [21]:
total_loss

tensor(1.7983)

In [ ]:
import torch.nn.functional as F

for name, m in model.named_modules():
    if isinstance(m, nn.Linear) | isinstance(m, Conv1D):
        if name.split('.')[-1] != 'lm_head':
            original = torch.load(opt_act_path+'/'+name+'.pt')
            quant = torch.load(quant_act_path+'/'+name+'.pt')
            